In [74]:
import numpy as np
from math import floor
from scipy.optimize import minimize ## linprog

In [75]:
def solution(fun, bounds, constraints):
    x_0 = []
    for i in range(len(bounds)):
        x_0.append(1)
    return minimize(fun, x_0, method='SLSQP', bounds = bounds, constraints = constraints)


In [76]:
def add_bounds(bounds, b_start, b_end):
    bound = [b_start, b_end]
    bounds.append(bound)


In [77]:
def add_constraints(constraints, type, constraint_function):
    constraint = {'type': type, 'fun': constraint_function}
    constraints.append(constraint) 

In [78]:
## Method to check if an array contains only integers

def x_is_integer(x):
    rem = 0
    for i in x:
        rem += i%1
    return rem == 0

In [79]:
## Method to find the index of an element with the largest decimal part in an array

def get_largest_dec_index(x):
    index = 0
    temp = 0

    for i in range(len(x)):
        r = x[i]%1
        if(r > temp):
            temp = r
            index = i
    return index

In [80]:
def round_array(x):
    ret = []
    for i in range(len(x)):
        ret.append(round(x[i],2))
    return ret

In [81]:
def floor_array(x):
    ret = []
    for i in range(len(x)):
        ret.append(floor(x[i]))
    return ret

In [82]:
## Method for solving maximization problems using branch and bound algorithm  

def BnB(fun, bounds, constraints):
    main_sol = solution(fun, bounds, constraints)
    x = round_array(main_sol.x)
    
    if(x_is_integer(x)):                                        ## stoping condition
        return {'x': x, 'solution': -fun(x)}                    ## an integer solution has been found
    UB = -fun(x)
    LB = -fun(floor_array(x))

    index = get_largest_dec_index(x)
    a = floor(x[index])
    b = a + 1
    
    ## Left branch of the recursion tree
    left_bounds = bounds.copy()
    left_bounds[index] = [left_bounds[index][0], a]
    sol1 = solution(fun, left_bounds, constraints)
    res1 = round(-sol1.fun,2) if (sol1.success or x_is_integer(sol1.x)) and (-sol1.fun <=UB and -sol1.fun >= LB) else -float('inf')  ## if potental solution is infeasble asign -infinity to it

    ## Right branch of the recursion tree
    right_bounds = bounds.copy()
    right_bounds[index] = [b, right_bounds[index][1]]
    sol2 = solution(fun, right_bounds, constraints)
    res2 = round(-sol2.fun,2) if (sol2.success or x_is_integer(sol2.x)) and (-sol2.fun <=UB and -sol2.fun >= LB) else -float('inf') ## if potental solution is infeasble asign -infinity to it


    if(res1>res2):                                                  ## deside the direction of therecursion 
        a = BnB(fun,left_bounds,constraints)                        ## find solution 
        if a["solution"] >= res2:                                   ## check if the soltion gives a greater result than the UB of other branch 
            return a                                                ## if yes return the solution 
        else:
            b = BnB(fun,right_bounds,constraints)                   ## get the solution for the other branch 
            return b if a["solution"] < b["solution"] else a        ## return the maximum of two solutions 

    else:
        b = BnB(fun,right_bounds,constraints)
        if b["solution"] >= res1:
            return b
        else:
            a = BnB(fun,left_bounds,constraints)
            return a if a["solution"] > b["solution"] else b

In [83]:
## Example 1
def f(x):
    x1 = x[0]
    x2 = x[1]

    return -(100*x1+150*x2)         ## Maximize f(x) == Minimiz -f(x)

In [84]:
def g1(x):
    x1 = x[0]
    x2 = x[1]

    return -(8000*x1+4000*x2-40000)      ## Minus needed for minimization 

In [85]:
def g2(x):
    x1 = x[0]
    x2 = x[1]

    return -(15*x1+30*x2-200  )          ## Minus needed for minimization 

In [86]:
bounds = []

add_bounds(bounds, 0, float('inf'))
add_bounds(bounds, 0, float('inf'))

constraints = []
add_constraints(constraints, 'ineq', g1)
add_constraints(constraints, 'ineq', g2)

print(BnB(f,bounds, constraints))

{'x': [1.0, 6.0], 'solution': 1000.0}


In [87]:
## Example 2

def f1(x):
    x1 = x[0]
    x2 = x[1]
    return -(17*x1+12*x2)

In [88]:
def g1_1(x):
    x1 = x[0]
    x2 = x[1]

    return -(10*x1+7*x2-40)

In [89]:
def g1_2(x):
    x1 = x[0]
    x2 = x[1]

    return -(x1+x2-5)


In [90]:
bounds1 = []

add_bounds(bounds1, 0, float('inf'))
add_bounds(bounds1, 0, float('inf'))

constraints1 = []
add_constraints(constraints1, 'ineq', g1_1)
add_constraints(constraints1, 'ineq', g1_2)

print(BnB(f1,bounds1, constraints1))

{'x': [4.0, 0.0], 'solution': 68.0}


In [91]:
#Example 3 (TaylorB page 13)

def f3(x):
    x1 = x[0]
    x2 = x[1]
    x3 = x[2]
    return -(5*x1+6*x2+4*x3)


In [92]:
def g3_1(x):
    x1 = x[0]
    x2 = x[1]
    x3 = x[2]
    return -(5*x1+3*x2+6*x3-20)

In [93]:
def g3_2(x):
    x1 = x[0]
    x2 = x[1]
    return -(x1+3*x2-12)

In [94]:
bounds3 = []

add_bounds(bounds3, 0, float('inf'))
add_bounds(bounds3, 0, float('inf'))
add_bounds(bounds3, 0, float('inf'))

constraints3 = []
add_constraints(constraints3, 'ineq', g3_1)
add_constraints(constraints3, 'ineq', g3_2)


print(BnB(f3,bounds3, constraints3))

{'x': [0.0, 4.0, 1.0], 'solution': 28.0}
